In [1]:
import csv
import math
import pandas as pd
import random
import gzip
import torch
from sklearn import metrics
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import gensim
import multiprocessing
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os
import argparse
import warnings
import torch.nn.functional as F
from torch import optim
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
import csv
from statsmodels.tsa.stattools import acf
from numpy import array
from numpy import hstack

c:\anaconda\envs\Deeplearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Functions

In [3]:
# split a multivariate sequence into samples
def Convert_format(data, n_steps_in):
    n0=np.where(data['PeriodsSepLastTwoNnZeroDemands']>0)[0][0]
    n1=n0+int((data.shape[0]-n0)*90/100)
    in_seq1=data['ZNZDemand'].values[n0:].reshape((-1, 1))
    in_seq2=data['LastQty'].values[n0:].reshape((-1, 1))
    in_seq3=data['WeekDay'].values[n0:].reshape((-1, 1))
    in_seq4=data['Interval'].values[n0:].reshape((-1, 1))
    in_seq5=data['PeriodsSepLastTwoNnZeroDemands'].values[n0:].reshape((-1, 1))
    in_seq6=data['Month'].values[n0:].reshape((-1, 1))
    # horizontally stack columns
    dataset = hstack((in_seq1, in_seq2,in_seq3,in_seq4,in_seq5,in_seq6))
    X = []
    for i in range(len(dataset)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + 1
        # check if we are beyond the dataset
        if (out_end_ix-1) > len(dataset):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = list(dataset[i:end_ix,1:]), list(dataset[end_ix-1:out_end_ix-1, 0])
        X.append([seq_x,seq_y])
    size=int((n1) / 5)
    train_data=X[:n1]
    test_data=X[n1:]
    calib_data = X[:4*size]
    valid_data = X[4*size:n1]
    return calib_data,valid_data,train_data,test_data

In [4]:
class dataset_load(Dataset):
    def __init__(self,xy=None):
        self.x_data=np.asarray([el[0] for el in xy],dtype=np.float32)
        self.y_data =np.asarray([el[1] for el in xy ],dtype=np.float32)
        self.x_data = torch.from_numpy(self.x_data)
        self.y_data = torch.from_numpy(self.y_data)
        self.len=len(self.x_data)
      

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [5]:
def make_confusion_matrix(cf,labels,pred,group_names=None,categories='auto',count=True,percent=True,cbar=True,xyticks=True,xyplotlabels=True,sum_stats=True,figsize=None,cmap='Blues',title=None):
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision, recall, thresholds = precision_recall_curve(labels, pred)
            Auc_PRC=metrics.auc(recall, precision)
            Auc_ROC=metrics.roc_auc_score(labels, pred)
            stats_text = "\n\nAU_PRC={:0.3f}\nAU_ROC={:0.3f}".format(Auc_PRC,Auc_ROC)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

# Network

In [6]:
class Deepnet(nn.Module):
    def __init__ (self,RNN,RNN_hidden_size,RNN_mean,RNN_sigma,layer_size,dropprob,n_features,Nout=1):
        super(Deepnet,self).__init__()
        self.RNN=RNN
        self.RNN_hidden_size=RNN_hidden_size
        self.RNN_sigma=RNN_sigma   
        self.layer_size=layer_size
        self.RNN_mean=RNN_mean   
        self.input_channels=n_features
        self.dropprob=dropprob
        self.out=Nout
        if self.RNN=='LSTM':
            self.rnn = nn.LSTM(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=False).to(device)
            self.FC_size= RNN_hidden_size
        elif self.RNN=='BiLSTM':
            self.rnn = nn.LSTM(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=True).to(device)
            self.FC_size= 2*RNN_hidden_size
        elif self.RNN=='GRU':
            self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=False).to(device)
            self.FC_size= RNN_hidden_size
        elif self.RNN=='BiGRU':
            self.rnn = nn.GRU(self.input_channels, RNN_hidden_size, num_layers=1, bidirectional=True).to(device)
            self.FC_size= 2*RNN_hidden_size
        for layer_p in self.rnn._all_weights:
            for p in layer_p:
                if 'weight' in p:
                    torch.nn.init.normal_(self.rnn.__getattr__(p),mean=RNN_mean,std=RNN_sigma)  
        self.dropout = torch.nn.Dropout(p=dropprob, inplace=False) #Dropout Layer (Dropout rate= p)
        #weights between LSTM or GRU layers and fully connected layer
        self.classifier = nn.Sequential(
            nn.Linear(self.FC_size,self.layer_size),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropprob, inplace=False),
            nn.Linear(self.layer_size, self.out),
            nn.Sigmoid())
    
    def forward(self,x):
        x=x.permute(1,0,2)
        output,_=self.rnn(x)
        if self.RNN=='BiLSTM' or self.RNN=='BiGRU':
            Normal_RNN=output[-1, :, :self.RNN_hidden_size]
            Rev_RNN=output[0, :, self.RNN_hidden_size:]
            x = torch.cat((Normal_RNN, Rev_RNN), 1)
            x=self.dropout(x)
        else:
            x = output[-1, :, :]
            x=self.dropout(x)
        x = self.classifier(x)
        return (x)

# Calibration

In [7]:
def logsampler(a,b):
        x=np.random.uniform(low=0,high=1)
        y=10**((math.log10(b)-math.log10(a))*x + math.log10(a))
        return y

In [8]:
def Calibration(RNN,w1,w0,calib_loader,valid_loader,n_features,metric):
    best_AUC = 0
    if verbose:
        print('Training on ',device)
    max_learning_steps = 500
    RNN_hidden_size_list = [20, 50, 80, 100]
    dropoutList = [0, 0.15, 0.3, 0.45]
    layer_size_list = [32, 64]
    learning_rate_list = [10**-5,10**-4,10**-3,10**-2]
    max_num_models = 40
    for number in range(max_num_models):
        if verbose:
            print('model {0} out of {1}'.format(number+1,max_num_models))
        # hyper-parameters
        RNN_hidden_size = random.choice(RNN_hidden_size_list)
        dropprob = random.choice(dropoutList)
        layer_size = random.choice(layer_size_list)
        learning_rate=random.choice(learning_rate_list)
        RNN_mean = random.choice([-1,1])*logsampler(10 ** -4, 1)
        RNN_sigma = logsampler(10 ** -4, 10 ** -2)
        model = Deepnet(RNN,RNN_hidden_size,RNN_mean,RNN_sigma,layer_size,dropprob,n_features).to(device)
        optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
        learning_steps = 0
        while learning_steps < max_learning_steps:
            loss_per_epoch = 0
            model.train()
            for batch_idx, (data1, target1, _, _) in enumerate(calib_loader):
                data = data1.to(device)
                target = target1.to(device)
                # Forward pass
                outputs = model(data)
                criterion = nn.BCELoss(reduction='mean',weight=((torch.abs((target)) * w1) - (torch.subtract(target,1) * w0)))
                loss = criterion(outputs, target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_per_epoch+=loss.item()
            learning_steps+=1
            if (learning_steps % 100 == 0):
                if verbose:
                    print("model trained for {0} epochs, loss per epoch is {1}.".format(learning_steps,loss_per_epoch/(batch_idx+1)))
                with torch.no_grad():
                    model.eval()
                    auc = []
                    for batch_idx, (data1, target1, _, _) in enumerate(valid_loader):
                        data = data1.to(device)
                        target = target1.to(device)
                        # Forward pass
                        output = model(data)
                        pred = output.cpu().detach().numpy().reshape(output.shape[0])
                        labels = target.cpu().numpy().reshape(output.shape[0])
                        if output.shape[0] > 60:
                            if (metric=='ROC'):
                                auc.append(metrics.roc_auc_score(labels, pred))
                            elif (metric=='PRC'):
                                precision, recall, _ = precision_recall_curve(labels, pred)
                                auc.append(metrics.auc(recall, precision))
                            else :
                                print('Choose proper metric')
                    AUC = np.mean(auc)
                    # print(AUC)
                    if AUC > best_AUC:
                        best_AUC = AUC
                        best_learning_steps = learning_steps
                        best_LearningRate = learning_rate
                        best_RNN_hidden_size=RNN_hidden_size
                        best_dropprob = dropprob
                        best_layer_size= layer_size
                        best_RNN_sigma = RNN_sigma
                        best_RNN_mean = RNN_mean
    if verbose:
        print('best_AUC=', best_AUC)
        print('best_learning_steps=', best_learning_steps)
        print('best_LearningRate=', best_LearningRate)
        print('best_dropprob=', best_dropprob)
        print('best_RNN_hidden_size=', best_RNN_hidden_size)
        print('best_layer_size=', best_layer_size)
        print('best_RNN_sigma=', best_RNN_sigma)
        print('best_RNN_mean=', best_RNN_mean)
    best_hyperparameters = {'best_learning_steps': best_learning_steps, 
                            'best_LearningRate': best_LearningRate,
                            'best_dropprob': best_dropprob, 
                            'best_RNN_hidden_size': best_RNN_hidden_size,
                            'best_layer_size': best_layer_size, 
                            'best_RNN_sigma': best_RNN_sigma,
                            'best_RNN_mean':best_RNN_mean}
    return best_hyperparameters

In [9]:
def Train_model(RNN,w1,w0,best_hyperparameters,train_loader,n_features,metric):
    best_learning_steps=8#best_hyperparameters['best_learning_steps']
    best_LearningRate=best_hyperparameters['best_LearningRate']
    best_RNN_hidden_size=best_hyperparameters['best_RNN_hidden_size']
    best_dropprob=best_hyperparameters['best_dropprob']
    best_RNN_mean=best_hyperparameters['best_RNN_mean']
    best_RNN_sigma=best_hyperparameters['best_RNN_sigma']
    best_layer_size=best_hyperparameters['best_layer_size']
    best_AUC=0
    best_threshold=0.5
    for number_models in range(5):
        model = Deepnet(RNN,best_RNN_hidden_size,best_RNN_mean,best_RNN_sigma,best_layer_size,best_dropprob,n_features).to(device)
        optimizer = torch.optim.Adam(model.parameters(),lr=best_LearningRate)
        learning_steps=0
        model.train()
        while learning_steps<best_learning_steps:
            for batch_idx, (data1, target1, _, _) in enumerate(train_loader):
                data = data1.to(device)
                target = target1.to(device)
                outputs = model(data)
                criterion = nn.BCELoss(reduction='mean',weight=((torch.abs((target)) * w1) - (torch.subtract(target,1) * w0)))
                loss = criterion(outputs, target)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            learning_steps+=1
        with torch.no_grad():
            model.eval()
            auc=[]
            threshold=[]
            for batch_idx, (data1, target1, _, _) in enumerate(train_loader):
                data = data1.to(device)
                target = target1.to(device)
                # Forward pass
                output = model(data)
                pred = output.cpu().detach().numpy().reshape(output.shape[0])
                labels=target.cpu().numpy().reshape(output.shape[0])
                if pred.shape[0]>60:
                    if (metric=='ROC'):
                        auc.append(metrics.roc_auc_score(labels, pred))
                        fpr, tpr, thresholds = roc_curve(labels, pred)
                        gmeans=np.sqrt(tpr*(1-fpr))
                        threshold.append(thresholds[np.argmax(gmeans)])
                    elif (metric=='PRC'):
                        precision, recall, thresholds = precision_recall_curve(labels, pred)
                        auc.append(metrics.auc(recall, precision))
                        gmeans=(2*precision*recall)/(precision+recall)
                        threshold.append(thresholds[np.argmax(gmeans)])
                    else :
                        print('Choose proper metric') 
            AUC_training=np.mean(auc)
            if verbose:
                print('AUC on training data for model ',number_models+1,' = ',AUC_training)
            if AUC_training>best_AUC:
                best_AUC=AUC_training
                best_threshold=np.mean(np.array(threshold),axis=0)
                best_model=model
    return best_model,best_threshold

In [10]:
def test_predict(best_model,test_loader):

    with torch.no_grad():
        best_model.eval()
        auc = []

        for batch_idx, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)

            # Forward pass
            output = best_model(data)
            pred = output.cpu().detach().numpy().reshape(output.shape[0])
            labels = target.cpu().numpy().reshape(output.shape[0])
            if output.shape[0] > 50:
                auc.append(metrics.roc_auc_score(labels, pred))

        AUC_test = np.mean(auc)
    return (labels,pred)

In [ ]:
evaluate_performance=True

# Product data path
path="./gen data/"
product="KS807 SOLUTION NETTOYAGE ECHANTILLONS"
DATA=pd.read_csv(path+product+".csv", sep=';',encoding = "ISO-8859-1")

# choose a number of time steps
Nout=1
Nin=1
metric='ROC'
# Data process
n_steps_in, n_steps_out = Nin, Nout
calib_data,valid_data,train_data,test_data=Convert_format(DATA,n_steps_in)
n_features=calib_data[0][0][0].shape[0]
batch_size=64
calib_dataset=dataset_load(calib_data)
valid_dataset=dataset_load(valid_data)
train_dataset=dataset_load(train_data)
test_dataset=dataset_load(test_data)
calib_loader = DataLoader(dataset=calib_dataset,batch_size=batch_size,shuffle=False)
valid_loader = DataLoader(dataset=valid_dataset,batch_size=batch_size,shuffle=False)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

# Generate Confusion matrix

In [ ]:
best_hyperparameters=Calibration(RNN,w1,w0,firsttrain_loader,firstvalid_loader,n_features,metric)
best_model,best_threshold=Train_model(RNN,w1,w0,best_hyperparameters,train_loader,n_features,metric)
labels,pred=test_predict(best_model,test_loader)
prediction_test=np.where(pred<best_threshold,0,pred)
prediction_test=np.where(prediction_test>best_threshold,1,prediction_test)
labels_train,pred_train=train_predict(best_model,alldata_loader)
prediction_train=np.where(pred_train<best_threshold,0,pred_train)
prediction_train=np.where(prediction_train>best_threshold,1,prediction_train)

In [ ]:
sns.set(font_scale=1.5)
cf = confusion_matrix(labels_train,prediction_train)
Matrix_labels = ["TN","FP","FN","TP"]
categories = ["N", "P"]
make_confusion_matrix(cf,labels_train,pred_train, group_names=Matrix_labels,categories=categories, cmap="binary")

In [ ]:
sns.set(font_scale=1.5)
cf = confusion_matrix(labels,prediction_test)
Matrix_labels = ["TN","FP","FN","TP"]
categories = ["N", "P"]
make_confusion_matrix(cf,labels,pred, group_names=Matrix_labels,categories=categories, cmap="binary")

# generate and save, in csv file, results for different w1 

In [ ]:
# Model process
header = ['product', 'w1','LSTM', 'BiLSTM', 'GRU','BiGRU']
RNN_list=['LSTM','BiLSTM','GRU','BiGRU']
w1_list=[1,DATA_Micro['ZNZDemand'].value_counts()[0]/DATA_Micro['ZNZDemand'].value_counts()[1]/2,DATA_Micro['ZNZDemand'].value_counts()[0]/DATA_Micro['ZNZDemand'].value_counts()[1]]
w0=1
for w1 in w1_list:
    AUC_list=[]
    for RNN in RNN_list:
        best_hyperparameters=Calibration(RNN,w1,w0,firsttrain_loader,firstvalid_loader,n_features,metric)
        best_model,best_threshold=Train_model(RNN,w1,w0,best_hyperparameters,train_loader,n_features,metric)
        auc=test_predict(best_model,test_loader)
        AUC_list.append(auc)
    dicti={
    'product':product,
    'w1':w1,
    'LSTM':AUC_list[0],
    'BiLSTM':AUC_list[1],
    'GRU':AUC_list[2],
    'BiGRU':AUC_list[3]
    }
    # saving results
    try:
        df=pd.read_csv('Bi_LSTM_GRU.csv')
        new=False
    except:
        new=True
    if new:
        with open('Bi_LSTM_GRU.csv','w') as fd:
            writer = csv.writer(fd)
            writer.writerow(header)
            writer = csv.DictWriter(fd, fieldnames=header)
            writer.writerow(dicti)
    else:
        with open('Bi_LSTM_GRU.csv','a',newline='') as fd:
            writer = csv.DictWriter(fd, fieldnames=header)
            writer.writerow(dicti)